Pan-Cancer

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder


raw_clinical_path = './data/TCGA-CDR-SupplementalTableS1.xlsx'
pancancer_save_path = './preprocessed_data/clinical.csv'

# Read clinical data
data = pd.read_excel(raw_clinical_path)

In [45]:
# Select the used features
data_used = data[['bcr_patient_barcode', 'type', 'gender', 'race', 'histological_type',
					'age_at_initial_pathologic_diagnosis', 'OS', 'OS.time']]
data_used.columns = ['id', 'cancer_type', 'gender', 'race', 'histological_type', 'age', 'event', 'event_time']


idx = data_used[data_used[['event', 'event_time']].isnull().T.any()].index
data_used.drop(labels=idx, inplace=True)
data_used.loc[data_used['race'] == '[Not Evaluated]', 'race'] = 'Na'
data_used.loc[data_used['race'] == '[Unknown]', 'race'] = 'Na'
data_used.loc[data_used['race'] == '[Not Available]', 'race'] = 'Na'

/tmp/ipykernel_11511/2560649441.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used.drop(labels=idx, inplace=True)


In [46]:
# categorical data and continuous data
data_id = data_used[['id']]
data_cate = data_used[['cancer_type', 'gender', 'race', 'histological_type']]
data_num = data_used[['age']]
target = data_used[['event', 'event_time']]

In [47]:
# Convert label to category type
data_cate['histological_type'] = data_cate['histological_type'].astype('str')
for col in data_cate.columns:
	data_cate[col] = LabelEncoder().fit_transform(data_cate[col])

for col in data_cate.columns:
	data_cate[col] = data_cate[col].astype('category')

/tmp/ipykernel_11511/1565323335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cate['histological_type'] = data_cate['histological_type'].astype('str')
/tmp/ipykernel_11511/1565323335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cate[col] = LabelEncoder().fit_transform(data_cate[col])
/tmp/ipykernel_11511/1565323335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [48]:
# Use 0 to fill NaN in age column
idx_num = data_num[data_num.isnull().T.any()].index
data_num.loc[idx_num, 'age'] = 0

/tmp/ipykernel_11511/1105121086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_num.loc[idx_num, 'age'] = 0


In [49]:
# Define the embedding size
embedded_cols = {n: len(col.cat.categories) for n,col in data_cate.items()}
embedding_sizes = [(n_categories, min(50, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
print(embedding_sizes)

[(33, 17), (2, 1), (6, 3), (145, 50)]


In [50]:
# concate the categorical data and continuous data
df = pd.concat([data_id, data_cate], axis=1)
df = pd.concat([df, data_num], axis=1)
df = pd.concat([df, target], axis=1)

df.to_csv(pancancer_save_path, header=False, index=False)

Kidney Cancer

In [ ]:
import pandas as pd


dataset_save_path = './preprocessed_data/clinical_kidney.csv'
dataset_index = [10, 11, 12]
# Bladder: [1], Breast: [2], Colorectal: [5, 23], Kidney: [10, 11, 12]

df_all = pd.read_csv(pancancer_save_path, header=None)
df_subset = df_all.loc[df_all[1].isin(dataset_index)]
df_subset.to_csv(dataset_save_path, header=False, index=False)
df_subset

,0,1,2,3,4,5,6,7
3765,TCGA-KL-8323,10,0,5,36,57.0,1.0,1158.0
3766,TCGA-KL-8324,10,0,5,36,67.0,0.0,4676.0
3767,TCGA-KL-8325,10,0,5,36,56.0,1.0,725.0
3768,TCGA-KL-8326,10,1,5,36,69.0,0.0,3322.0
3769,TCGA-KL-8327,10,0,5,36,46.0,0.0,4169.0
...,...,...,...,...,...,...,...,...
4699,TCGA-Y8-A898,12,1,5,38,69.0,0.0,475.0
4700,TCGA-Y8-A8RY,12,1,5,38,63.0,0.0,769.0
4701,TCGA-Y8-A8RZ,12,1,5,38,55.0,0.0,205.0
4702,TCGA-Y8-A8S0,12,1,5,38,58.0,0.0,183.0
